# FastAPI Lecture 2: HTTP Methods, Parameters, and Routers

Welcome to the second lecture on FastAPI! In this session, we will explore fundamental concepts for building more interactive and organized APIs:

1.  **HTTP Methods:** Understanding how clients communicate actions (GET, POST, PUT, DELETE).
2.  **Path & Query Parameters:** Learning how to receive data through the URL.
3.  **FastAPI Routers:** Organizing your API endpoints into logical modules.

**Important Note:** All the code examples for this lecture are provided in separate `.py` files. This notebook contains explanations and instructions on how to run those files.

## Understanding HTTP Methods

When building web APIs, we use different **HTTP methods** (also called verbs) to tell the server what kind of action we want to perform on a specific resource (like a piece of data). Think of them as instructions for the server.

FastAPI makes it easy to handle these methods using decorators. The most common methods you'll encounter are:

*   **GET:** This is used to request data from the server. When you visit a webpage in your browser, you're usually making a GET request. In API terms, it's used to retrieve information, like getting details about a specific user or a list of items.

*   **POST:** This method is typically used to send data to the server to create a new resource. For example, when you fill out a sign-up form or submit a new blog post, you're often sending a POST request.

*   **PUT:** This method is generally used to update an existing resource completely. If you want to change all the details of a user profile, you might use a PUT request, sending the full updated profile.

*   **DELETE:** As the name suggests, this method is used to request the server to delete a specific resource. For instance, deleting a user account or removing an item from a list.

FastAPI provides decorators like `@app.get("/path")`, `@app.post("/path")`, `@app.put("/path")`, and `@app.delete("/path")` to handle these different request types easily.

### Running the HTTP Method Examples

We have separate files demonstrating each method. You need to run each file individually from your terminal.

**1. GET Example ([`get_example.py`](./get_example.py))**

   - Open your terminal, navigate to the directory containing the files, and run:
     ```bash
     python get_example.py
     ```
   - Open your web browser and go to `http://127.0.0.1:8000`. You should see the welcome message.

**2. POST Example ([`post_example.py`](./post_example.py))**

   - Run the file (it uses port 8001):
     ```bash
     python post_example.py
     ```
   - Since POST requests send data, you can't easily test this with just a browser URL. Use the interactive API docs:
     - Go to `http://127.0.0.1:8001/docs` in your browser.
     - Find the POST `/items/` endpoint, click "Try it out", fill in the example request body, and click "Execute".

**3. PUT Example ([`put_example.py`](./put_example.py))**

   - Run the file (it uses port 8002):
     ```bash
     python put_example.py
     ```
   - Similar to POST, use the API docs at `http://127.0.0.1:8002/docs`.
   - Find the PUT `/items/{item_id}` endpoint. Try updating item `1` by entering `1` in the `item_id` field, providing the request body, and executing.

**4. DELETE Example ([`delete_example.py`](./delete_example.py))**

   - Run the file (it uses port 8003):
     ```bash
     python delete_example.py
     ```
   - Use the API docs at `http://127.0.0.1:8003/docs`.
   - Find the DELETE `/items/{item_id}` endpoint. Try deleting item `1` by entering `1` for `item_id` and executing.
   - You can then use the GET `/items/` endpoint in the same docs to see that item 1 is gone.

## Working with Path Parameters

Sometimes, you need to identify a specific item or resource directly in the URL path. For example, if you want to get information about a user with ID `123`, your URL might look like `/users/123`. That `123` part is a **path parameter**.

In FastAPI, you define path parameters using curly braces `{}` directly in the path string of your decorator (e.g., `@app.get("/items/{item_id}")`). FastAPI automatically recognizes this and lets you use the parameter name (like `item_id`) as an argument in your path operation function. You can also specify the data type for the parameter (like `int` or `str`) using Python type hints, and FastAPI will handle data validation for you.

For example:
```python
# From path_param_example.py
@app.get("/users/{user_id}")
def get_user_by_id(user_id: int):
    # ... function body ...
```
Here, `user_id` is a path parameter expected to be an integer.

### Running the Path Parameter Example

(Associated file: [`path_param_example.py`](./path_param_example.py))

   - Run the file (it uses port 8004):
     ```bash
     python path_param_example.py
     ```
   - Test it by visiting URLs with different IDs in your browser:
     - `http://127.0.0.1:8004/users/1`
     - `http://127.0.0.1:8004/users/2`
     - `http://127.0.0.1:8004/users/99` (This should give a 'Not Found' error).
   - You can also explore this in the API docs: `http://127.0.0.1:8004/docs`

## Using Query Parameters

**Query parameters** are another way to send data to the server, but they are not part of the main URL path. Instead, they appear at the end of the URL after a question mark `?`, as key-value pairs separated by ampersands `&`. They are often used for filtering, sorting, or pagination.

Example URL with query parameters: `/items?skip=0&limit=10`
Here, `skip` and `limit` are query parameters.

In FastAPI, any function parameter declared in your path operation function that is *not* part of the path parameters is automatically interpreted as a query parameter. You can define their types and default values just like regular Python function arguments.

For example:
```python
# From query_param_example.py
from typing import Union

@app.get("/items/")
def read_items(skip: int = 0, limit: int = 10, q: Union[str, None] = None):
    # ... function body ...
```
In this example, `skip`, `limit`, and `q` are query parameters. `skip` and `limit` have default values, while `q` is optional (can be `None`). FastAPI handles extracting these values from the URL if they are provided.

### Running the Query Parameter Example

(Associated file: [`query_param_example.py`](./query_param_example.py))

   - Run the file (it uses port 8005):
     ```bash
     python query_param_example.py
     ```
   - Test different query combinations in your browser:
     - `http://127.0.0.1:8005/items/` (Uses defaults)
     - `http://127.0.0.1:8005/items/?skip=2&limit=2` (Pagination)
     - `http://127.0.0.1:8005/items/?q=ba` (Filtering)
     - `http://127.0.0.1:8005/items/?limit=3&q=u` (Filtering and pagination)
   - Explore the options in the API docs: `http://127.0.0.1:8005/docs`

## Organizing Your Code with FastAPI Routers

As your API grows larger, putting all your path operations (like `@app.get`, `@app.post`, etc.) into a single file can become messy and hard to manage. Imagine having dozens or even hundreds of endpoints in one file! To keep things organized, FastAPI provides a powerful feature called **APIRouter**.

Think of an `APIRouter` as a mini-FastAPI application. You can define path operations, dependencies, and even other routers within it, just like you do with the main `FastAPI` app. The key benefit is that you can group related endpoints together in separate Python modules (files).

For example, you might have one router for handling user-related operations (like creating users, getting user details) and another router for handling item-related operations (like adding items, listing items). This makes your codebase much cleaner and easier to navigate.

To use a router, you first import `APIRouter` from `fastapi`. Then, you create an instance of it, similar to how you create your main `FastAPI` instance:

```python
# From router_items.py
from fastapi import APIRouter

router = APIRouter()

@router.get("/")
def read_items():
    # ... function body ...
```

Notice how we use `@router.get` instead of `@app.get`. You define your path operations on the router instance.

Once you have defined your path operations in a separate file using an `APIRouter`, you need to tell your main FastAPI application to include this router. You do this using the `app.include_router()` method in your main application file (`main.py` in our example):

```python
# From router_main.py
from fastapi import FastAPI
from routers import items # Import the router module

app = FastAPI()

# Include the router with a prefix and tags
app.include_router(items.router, prefix="/items", tags=["items"])
```

When you include a router, you can also specify a `prefix` (like `/items`) which will be added before all paths defined in that router, and `tags` which help group the endpoints in the automatic API documentation (like Swagger UI).

Using routers is a fundamental practice for building maintainable and scalable FastAPI applications.

### Running the Router Example

This example involves two files: [`router_main.py`](./router_main.py) (main application) and [`router_items.py`](./router_items.py) (items router).

   - **Important:** Both files are in the current directory. Run the main application file (`router_main.py`):


   - **Important:** Navigate your terminal *inside* the `router_example` directory:
     ```bash
     ```
   - Run the main application file (it uses port 8006):
     ```bash
     python router_main.py 
     # Or use: uvicorn router_main:app --reload --port 8006
     ```
   - Test the endpoints:
     - Main app root: `http://127.0.0.1:8006/`
     - Items router root: `http://127.0.0.1:8006/items/`
     - Specific item via router: `http://127.0.0.1:8006/items/item1`
   - Check the API docs (`http://127.0.0.1:8006/docs`) and notice how the endpoints are grouped under the "items" tag.

## Conclusion

In this lecture, we covered essential FastAPI concepts:

- How to handle different **HTTP methods** (GET, POST, PUT, DELETE) to define the actions your API can perform.
- How to use **path parameters** to identify specific resources in the URL.
- How to use **query parameters** for optional data like filtering and pagination.
- How to organize your code effectively using **APIRouter**.

These building blocks are crucial for creating well-structured and functional web APIs with FastAPI. Practice running these examples and experiment with them!